In [1]:
##### PTT 股票版 

### 取得首頁 source code
import requests
from bs4 import BeautifulSoup as bs 

url = "https://www.ptt.cc/bbs/Stock/index.html"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

In [2]:
### 抓取首頁文章連結
links = []
for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        print(url,title)

https://www.ptt.cc/bbs/Stock/M.1636425256.A.39B.html [標的] 4961 天鈺 多
https://www.ptt.cc/bbs/Stock/M.1636425693.A.235.html [新聞] 投顧：基本面強勁 長榮有望過7月高點223
https://www.ptt.cc/bbs/Stock/M.1636425790.A.045.html Re: [請益] 菜雞請問存股族除息怎麼操作?


In [3]:
### 抓取 分頁文章 連結

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))
        
print("Done.")

https://www.ptt.cc/bbs/Stock/index4999.html is ok.
https://www.ptt.cc/bbs/Stock/index4998.html is ok.
https://www.ptt.cc/bbs/Stock/index4997.html is ok.
https://www.ptt.cc/bbs/Stock/index4996.html is ok.
https://www.ptt.cc/bbs/Stock/index4995.html is ok.
Done.


In [15]:
### 抓取文章本文 source code 
url = links[11]
print(url)

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res2 = requests.get(url,headers=headers)
soup2 = bs(res2.text,"lxml")


https://www.ptt.cc/bbs/Stock/M.1636419064.A.3AD.html


In [16]:
### 抓取本文的 作者 , 看板 , 標題 , 時間 
span_tags = soup2.select("div#main-content span.article-meta-value")

# 作者
author = span_tags[0].text
print("author :",author)

# 看板
category = span_tags[1].text
print("category :",category)

# 標題
title = span_tags[2].text
print("title :",title)

# 時間
time = span_tags[3].text
print("time :",time)

# span_tags

author : fgf (今年不太冷)
category : Stock
title : [請益] 菜雞請問存股族除息怎麼操作?
time : Tue Nov  9 08:51:02 2021


In [101]:
### 抓取本文的 回應

#############
# 為了標籤拔除 , 故先抓取 回應資料
# 加入拔除標籤動作
def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


push_tags = soup2.select("div#main-content div.push")
resp_data = []

if len(push_tags) >0:
    
    for ele in push_tags:
        ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤
        
        resp = get_resp_data(ele)
        
        resp_data.append(resp)
    
# print(resp_data)

In [100]:
### 內容
# print(soup2.select("div#main-content")[0].text)

# 1. 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2

def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup 



soup2 = remove_dirty_tag(soup2)

# 2. 抓取文字
content = soup2.select("div#main-content")[0].text.strip()
# print("content:", content)

In [95]:
### 組合程式
import requests
from bs4 import BeautifulSoup as bs 


# 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2
def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup

def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


def get_data(soup,url):
    ### 抓取本文的 作者 , 看板 , 標題 , 時間 
    span_tags = soup.select("div#main-content span.article-meta-value")

    # 作者
    author = span_tags[0].text

    # 看板
    category = span_tags[1].text

    # 標題
    title = span_tags[2].text

    # 時間
    time = span_tags[3].text

    ### 抓取本文的 回應

    #############
    # 為了標籤拔除 , 故先抓取 回應資料
    # 加入拔除標籤動作
    push_tags = soup.select("div#main-content div.push")
    resp_data = []

    if len(push_tags) >0:

        for ele in push_tags:
            ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤

            resp = get_resp_data(ele)

            resp_data.append(resp)
            
    ### 內容
    soup = remove_dirty_tag(soup)
    content = soup.select("div#main-content")[0].text.strip()
    
    
    ### 包成 dict return 回去
    return {
        "author" : author,
        "category" : category,
        "title" : title,
        "time" : time,
        "resp_data" : resp_data,
        "content" : content,
        "url" : url
    }



### main 程式

## 取得首頁 source code
base_url = "https://www.ptt.cc/bbs/Stock/index.html"

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

# res = sess.get(base_url,headers=headers)    # 後半段
res = requests.get(base_url,headers=headers)
soup = bs(res.text,"lxml")

print("* 取得首頁 source code 完成！")


## 抓取首頁文章連結
links = []

for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        links.append(url)
        
print("* 取得首頁文章連結 完成！")


        
## 抓取 分頁文章 連結
for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
#     res = sess.get(previous_link,headers=headers)  # 後半段
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))
        
print("* 抓取分頁文章連結 完成！")


## 抓取文章本文 source code + 資料清洗

dataList = []

for url in links[:10]:
    res2 = requests.get(url,headers=headers)
#     res2 = sess.get(url,headers=headers)   # 後半段
    soup2 = bs(res2.text,"lxml")
    data = get_data(soup2,url)
    
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("* 抓取本文資料 完成！")


print("Done.")

* 取得首頁 source code 完成！
* 取得首頁文章連結 完成！
https://www.ptt.cc/bbs/Gossiping/index39009.html is ok.
https://www.ptt.cc/bbs/Gossiping/index39008.html is ok.
https://www.ptt.cc/bbs/Gossiping/index39007.html is ok.
https://www.ptt.cc/bbs/Gossiping/index39006.html is ok.
https://www.ptt.cc/bbs/Gossiping/index39005.html is ok.
* 抓取分頁文章連結 完成！
https://www.ptt.cc/bbs/Gossiping/M.1636430219.A.177.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430265.A.8DA.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430270.A.191.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430279.A.DB3.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430280.A.BFD.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430374.A.1F9.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430380.A.0BC.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430421.A.153.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430475.A.58A.html is ok.
https://www.ptt.cc/bbs/Gossiping/M.1636430483.A.43A.html is ok.
* 抓取本文資料 完成！
Done.


In [ ]:
ele = dataList[0]
for key in ele:
    print(key , ":" , ele[key])

In [96]:
### 資料落地
import os

for data in dataList:

    file_name = data["title"]
    
    # 檢查 sample 資料夾是否存在
    # 若不存在 , 則新建立一個
    if not os.path.exists("sample"):
        os.mkdir("sample")

    with open("sample/{}.txt".format(file_name),"w") as out_file:

        record = ""

        # 除回應外 , 其餘資料拼接
        for key in data:
            if key != "resp_data": record += "{}: {}\n".format(key,data[key])
                
        # 回應分隔線
        record += ("="*80)
        record += "\n"
        
        # 回應資料
        resp = ""
        for ele in data["resp_data"]:
            resp += "{},{},{},{}\n".format(ele["tag"],ele["author"],ele["content"],ele["time"])
            
        record += resp
        out_file.write(record)

In [ ]:
### 程式轉成 .py 檔

In [ ]:
##########################################################################################

In [ ]:
##########################################################################################

In [ ]:
##########################################################################################

In [99]:
##### PTT 八卦板
import requests
from bs4 import BeautifulSoup as bs 

base_url = "https://www.ptt.cc/bbs/Gossiping/index.html"

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res4 = requests.get(base_url,headers=headers)
soup4 = bs(res4.text,"lxml")
# soup4

In [97]:
## 同意 18 歲驗證
api = "https://www.ptt.cc/ask/over18"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

payload = {
    "from": "/bbs/Gossiping/index.html",
    "yes" : "yes"
}

res3 = requests.post(api,headers=headers,data=payload)
# print(res3.text)
# soup

In [98]:
### 使用 session 紀錄狀態


## 1. 建立 session 
sess = requests.session()
 
## 2. 同意 18 歲驗證 (如同 會員登入)
api = "https://www.ptt.cc/ask/over18"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

payload = {
    "from": "/bbs/Gossiping/index.html",
    "yes" : "yes"
}

# 改用 sess 發 requests 
# res3 = requests.post(api,headers=headers,data=payload)
res3 = sess.post(api,headers=headers,data=payload)


## 3. 再用 sess 向八卦版爬取文章
base_url = "https://www.ptt.cc/bbs/Gossiping/index.html"

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}


# stateful session 發 requests (已紀錄 over18)
res4 = sess.get(base_url,headers=headers)

# stateless 發 requests  (未紀錄 over18)
# res4 = requests.get(base_url,headers=headers)


soup4 = bs(res4.text,"lxml")
# soup4

# soup